
# Extração de Tópicos com Latent Semantic Analysis (LSA)


Exemplos oferecidos pelo próprio scikit-learn no seguinte link:

* https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.TruncatedSVD.html




# Novo QGS

Iremos adotar a partir de agora um novo QGS, com 52 artigos relacionados a mineração de texto em auxílio à revisões sistemáticas da literatura, encontrados [aqui](Files-QGS/QGS-ia-pdf). Também está disponível uma planilha [aqui](Files-QGS/QGS.ods) com todos os nomes destes arquivos e se estes estão localizados na base de dados Scopus ou não. A principio, utilizaremos um dataset do QGS em formato .txt que possui apenas o título, palavras-chave e o resumo destes artigos, localizado [aqui](Files-QGS/QGS-ia-txt/metadata/).

Importante salientar que dos 52 arquivos utilizados como QGS **46** estão presentes na base de dados Scopus (Ao se procurar pelo titulo destes, são encontrados com facilidade).

Abaixo segue um código exemplo de como foi acessado tal base e efetuado novamente os testes realizados com o QGS anterior, alterando apenas os valores de number_topics e number_words.

In [5]:
# encoding: utf-8
from __future__ import print_function
from time import time
from sklearn.datasets import load_files
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import Pipeline
from sklearn.datasets import fetch_20newsgroups

number_topics = 2
number_words = 5
max_document_frequency = 1.0
min_document_frequency = 0.4
ngram = (1, 3)
max_features = None

iterations = 5000

# Imprime os tópicos com as palavras em ordem
def print_top_words(model, feature_names, number_words):
    for topic_index, topic in enumerate(model.components_):
        message = "Topic %d: " % (topic_index + 1)
        message += "{"
        message += " - ".join([feature_names[i]
                               for i in topic.argsort()[:-number_words - 1:-1]])
        message += "}\n"
        print(message)

def print_string(model, feature_names, number_words):
    message = ("TITLE-ABS-KEY(")
    for topic_index, topic in enumerate(model.components_):
        message += "(\""
        message += "\" AND \"".join([feature_names[i]
                               for i in topic.argsort()[:-number_words - 1:-1]])
        message += "\")"
        if topic_index < number_topics - 1:
            message += " OR "
        else:
            message += ""
    message += ")"
    print(message)


# Carrega o dataset de treinamento
files = load_files(container_path = '/home/fuchs/Documentos/MESTRADO/Masters/Files-QGS/QGS-ia-txt/metadata', encoding="iso-8859-1")

# Usa tf-idf para o NMF.
tfidf_vectorizer = TfidfVectorizer(max_df = max_document_frequency,
                                   min_df = min_document_frequency,
                                   ngram_range= ngram,
                                   max_features = max_features,
                                   stop_words='english')

tfidf = tfidf_vectorizer.fit_transform(files.data)

# Salva os nomes das palavras em um dicionário
dic = tfidf_vectorizer.get_feature_names()

# SVD para reduzir a dimensionalidade
svd_model = TruncatedSVD(n_components = number_topics,
                         algorithm = 'randomized',
                         n_iter = iterations)

# Pipeline do tf-idf + SVD, fit e aplicando nos arquivos
svd_transformer = Pipeline([('tfidf', tfidf_vectorizer), ('svd', svd_model)])

svd_transformer.fit_transform(files.data)

# Imprime os (number_topics) tópicos com as (number_words) palavras
print_top_words(svd_model, dic, number_words)

print_string(svd_model, dic, number_words)


Topic 1: {systematic - reviews - review - learning - systematic reviews}

Topic 2: {mining - process - text - literature - study}

TITLE-ABS-KEY(("systematic" AND "reviews" AND "review" AND "learning" AND "systematic reviews") OR ("mining" AND "process" AND "text" AND "literature" AND "study"))


## TESTES v.1

Iremos testar o algoritmo com a lógica em que o AND fica entre as palavras do mesmo tópico e OR entre os tópicos, realizado sobre a base de dados apenas dos metadados (apenas título, abstract e keywords), uma vez que foi a que retornou os melhores resultados no LDA.

-------------------------------------------------

### 1° Teste: 2 Tópicos e 5 palavras

(Código apresentado abaixo)

**Resultado**

Topic 1: {systematic - reviews - review - learning - systematic reviews}

Topic 2: {mining - process - text - literature - study}

String: TITLE-ABS-KEY(("systematic" AND "reviews" AND "review" AND "learning" AND "systematic reviews") OR ("mining" AND "process" AND "text" AND "literature" AND "study"))

Busca: 4,619 documentos encontrados 

-------------------------------------------------

### 2° Teste: 3 Tópicos e 5 palavras

(Código apresentado abaixo)

**Resultado**

Topic 1: {systematic - reviews - review - learning - systematic reviews}

Topic 2: {mining - process - text - literature - study}

Topic 3: {study - evidence - evidence based - literature - based}

String: TITLE-ABS-KEY(("systematic" AND "reviews" AND "review" AND "learning" AND "systematic reviews") OR ("mining" AND "process" AND "text" AND "literature" AND "study") OR ("study" AND "evidence" AND "evidence based" AND "literature" AND "based"))

Busca: 28,073 documentos encontrados